In [2]:
import torch
import triton
import triton.language as tl

In [3]:
@triton.jit 
def clipper(inptr,outptr,maxv,minv,n,blocksize:tl.constexpr):
    pid=tl.program_id(0)
    block_start=blocksize* pid
    offsets=block_start+ tl.arange(0,blocksize)
    mask=offsets<n
    val=tl.load(inptr+offsets,mask=mask)
    val=tl.where(val>maxv,maxv,val)
    val=tl.where(val<minv,minv,val)
    tl.store(outptr+offsets,val,mask=mask)

In [12]:
def test():
    data = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], device='cuda', dtype=torch.float32)
    n=len(data)
    b=torch.zeros_like(data)
    blocksize=32
    nofoblocks=triton.cdiv(n,blocksize)
    clipper[(nofoblocks,)](data,b,8,2,n,blocksize)
    print(b)

In [13]:
if __name__=="__main__":
    test()

tensor([2., 2., 3., 4., 5., 6., 7., 8., 8., 8.], device='cuda:0')
